In [1]:
import os
import pandas as pd
import numpy as np
import re

#определение региона по страны
from countryinfo import CountryInfo #https://github.com/porimol/countryinfo

# Гипотеза:  
___Экономика государств с развитой государственной поддержкой инновационных предприятий (инновационной системы) растёт интенсивнее.___

Для этого, выбираю следующие показатели:  

___1.1.	Political environment (интегральный покозатель)___  
1.1.1.	Political stability and absence of violence/terrorism  
1.1.2.	Government effectiveness  
1.1.3.	Press freedom  

___1.2.	Regulatory environment (интегральный покозатель)___  
1.2.1.	Regulatory quality  
1.2.2.	Rule of law  
1.2.3.	Cost of redundancy dismissal  

___1.3.	Business environment (интегральный покозатель)___  
1.3.1.	Ease of starting a business  
1.3.2.	Ease of resolving insolvency  
1.3.3.	Ease of paying taxes  

2.1.1.	Expenditure on education  
2.1.2.	Public expenditure on education per pupil  

2.1.5.	Pupil-teacher ratio, secondary  

2.2.1.	Tertiary enrolment  
2.2.2.	Graduates in science and engineering  

2.3.1.	Researchers  
2.3.2.	Gross expenditure on R&D (GERD)  

3.1.3.	Government's online service  

3.2.1.	Electricity output  
3.2.3.	Logistics performance  

3.3.2.	Environmental performance  


___4.1.	Credit (интегральный покозатель)___  
4.1.1.	Ease of getting credit  
4.1.2.	Domestic credit to private sector  
4.1.3.	Microfinance institutions' gross loan portfolio  

4.2.1.	Ease of protecting investors  

4.2.4.	Venture capital deals  

4.3.2.	Market access for non-agricultural exports  
4.3.3.	Intensity of local competition  

5.1.2.	Firms offering formal training  

___5.2.	Innovation linkages (интегральный покозатель)___   
5.2.1.	University/industry research collaboration  
5.2.2.	State of cluster development  
5.2.3.	GERD financed by abroad  
5.2.4.	Joint venture/strategic alliance deals  
5.2.5.	Patent families filed in at least three offices  

5.3.1.	Royalties and license fees payments (% of service imports)  
5.3.4.	Foreign direct investment net inflows  

6.1.2.	Patent Cooperation Treaty resident applications  

6.3.4.	Foreign direct investment net outflows

Я выбрал 37 показателей, которые могут характеризовать государственную поддержку и 5 интегральных позателей (вероятно они могут быть полезны для свёртки)

In [2]:
list_fe = ['Political stability and absence of violence/terrorism',
           'Government effectiveness',
           'Press freedom','Regulatory quality',
           'Rule of law', 'Cost of redundancy dismissal',
           'Ease of starting a business',
           'Ease of resolving insolvency',
           'Ease of paying taxes',
           'Expenditure on education',
           'Public expenditure on education per pupil',
           'Pupil-teacher ratio, secondary',
           'Tertiary enrolment',
           'Graduates in science and engineering','Researchers',
           'Gross expenditure on R&D (GERD)',
           "Government's online service",
           'Electricity output',
           'Logistics performance',
           'Environmental performance',
           'Ease of getting credit',
           'Domestic credit to private sector',
           "Microfinance institutions' gross loan portfolio",
           'Ease of protecting investors',
           'Venture capital deals',
           'Market access for non-agricultural exports',
           'Intensity of local competition',
           'Firms offering formal training',
           'University/industry research collaboration',
           'State of cluster development',
           'GERD financed by abroad',
           'Joint venture/strategic alliance deals',
           'Patent families filed in at least three offices',
           'Royalties and license fees payments (% of service imports)',
           'Foreign direct investment net inflows',
           'Patent Cooperation Treaty resident applications',
           'Foreign direct investment net outflows']

#интегральные показатели
list_intfe = ['Political environment', 'Regulatory environment', 'Business environment', 'Credit', 'Innovation linkages']

In [3]:
data_dir = 'C:\\Users\\dserp\\ML\\Degree_ML\\full'
file_list = os.listdir(data_dir)

## Формирую датасет

In [4]:
data = {} #словарь датасетов с показателми без интегральных

Предобработка

### Переименоваю названия стран, для дальнейшего распознования региона

In [5]:
rename_dict = {
    'Bolivia (Plurinational State of)' : 'Bolivia',
    'Bolivia, Plurinational St.' : 'Bolivia',
    'Bolivia, Plurinational State of' : 'Bolivia',
    'Bahamas' : 'Commonwealth of The Bahamas',
    'Bahamas, Commonwealth of the' : 'Commonwealth of The Bahamas',
    'Bahamas, Commonwealth of' : 'Commonwealth of The Bahamas',
    'Brunei Darussalam' : 'Nation of Brunei',
    'Brunei' : 'Nation of Brunei',
    'Cabo Verde' : 'Cape Verde',
    'Congo, Democratic Republic of the': 'Republic of the Congo',
    'Congo' : 'Republic of the Congo',
    "Cote d'Ivoire" : 'Ivory Coast',
    "Côte d'Ivoire" : 'Ivory Coast',
    'Cote dIvoire' : 'Ivory Coast',
    'Czech Republic (the)' : 'Czech Republic',
    'Dominican Republic (the)' : 'Dominican Republic',
    'Iran (Islamic Republic of)' : 'Iran',
    'Iran, Islamic Rep.' : 'Iran',
    'Iran, Islamic Republic of' : 'Iran',
    'Gambia' : 'Republic of The Gambia',
    'Guinea-Bissau' : 'Republic of Guinea-Bissau',
    'Guinea Bissau' : 'Republic of Guinea-Bissau',
    'Guinea-Bissau, Republic of': 'Republic of Guinea-Bissau',
    'Guinea':'Republic of Guinea-Bissau',
    'Holy See (Vatican City State)' : 'Vatican',
    'Hong Kong (China)' : 'Hong Kong',
    'Hong Kong, China' : 'Hong Kong',
    "Korea, Democratic People's Republic of" : "Democratic People's Republic of Korea",
    'Korea, Rep.' : 'Republic of Korea',
    'Republic of Korea (the)' : 'Republic of Korea',
    'Korea, Republic of' : 'Republic of Korea',
    'Moldova, Rep.' : 'Moldova',
    'Moldova, Republic of' : 'Moldova',
    'Micronesia, Federated States of' : 'Federated States of Micronesia',
    'Micronesia' : 'Federated States of Micronesia',
    'Republic of Moldova (the)' : 'Moldova',
    'North Macedonia' : 'Republic of Macedonia',
    'TFYR Macedonia' : 'Republic of Macedonia',
    'The Former Yugoslav Republic (FYR) of Macedonia' : 'Republic of Macedonia',
    'Timor-Leste' : 'Democratic Republic of Timor-Leste',
    'Netherlands (the)' : 'Netherlands',
    'Niger (the)' : 'Niger',
    'Russian Federation (the)' : 'Russian Federation',
    'Sudan (pre-secession)' : 'Sudan',
    'Sao Tome and Principe' : "Democratic Republic of São Tomé and Príncipe",
    'Tanzania, United Rep.' : 'Tanzania',
    'Tanzania, United Republic of' : 'Tanzania',
    'United Republic of Tanzania (the)' : 'Tanzania',
    'Taiwan, Province of China' : 'Taiwan',
    'United Arab Emirates (the)' : 'United States of America',
    'United States of America (the)' : 'United States of America',
    'United Kingdom (the)' : 'United Kingdom',
    'Venezuela, Bolivarian Rep.' : 'Venezuela',
    'Venezuela, Bolivarian Republic of' : 'Venezuela',
    'Viet Nam' : 'Vietnam'}

In [6]:
int_fe = {}

for file in file_list:
    df_temp = pd.read_csv(os.path.join(data_dir, file))
        
    #Переименовываю столбцы 
    df_temp.rename(columns = {list(df_temp.iloc[:, [0]])[0] : re.sub('\D','', file), 
                            list(df_temp.iloc[:, [1]])[0] : 'Indicator'}, inplace = True)
                   
    #переименование стран
    df_temp.rename(columns = rename_dict, inplace = True)
    
    #удаляю дупликаты
    df_temp = df_temp.loc[:,~df_temp.columns.duplicated()]
    
    #удаляю строку 'Indicator' (индекс 0) и 'Index' (индекс 4 или 5) - они содержат только Nan (видимо для красоты таблички, для открытия в Excel)
    a = df_temp.loc[df_temp[list(df_temp.iloc[:, [0]])[0]] == 'Index'].index
    df_temp.drop([0, a[0]], inplace = True)

    #в датафрейме есть значения '  '(и не числовые значения) - несколько пробелов, заменяю их на None
    df_temp.iloc[:,3:] = df_temp.iloc[:,3:].replace(to_replace = '[\D]', value = np.nan, regex = True)
    
    #заполняю пропуски нулями
    df_temp.fillna(0, inplace = True)
    
    #отсекаю всё лишнее
    df_temp = df_temp.loc[:, :'Zimbabwe']
                
    indicators_temp = pd.DataFrame(data = None, columns = list(df_temp))
    for i in list_fe:
        indicators_temp = pd.concat([indicators_temp, df_temp.loc[df_temp['Indicator']== i]])
    indicators_temp.drop(list(df_temp.iloc[:, [2]])[0], axis = 1, inplace = True)
    
    #добавляю показатели в датасет
    data[re.sub('\D','', file)] = indicators_temp
    
    indicators_temp2 = pd.DataFrame(data = None, columns = list(df_temp))
    for i in list_intfe:
        indicators_temp2 = pd.concat([indicators_temp2, df_temp.loc[df_temp['Indicator']== i]])
    indicators_temp2.drop(list(df_temp.iloc[:, [2]])[0], axis = 1, inplace = True)
    
    int_fe['int_fe_' + re.sub('\D','', file)] = indicators_temp2

In [7]:
 int_fe.keys()

dict_keys(['int_fe_2013', 'int_fe_2014', 'int_fe_2015', 'int_fe_2016', 'int_fe_2017', 'int_fe_2018', 'int_fe_2019'])

In [8]:
int_fe['int_fe_2014'].drop('Afghanistan', axis = 1, inplace = True)
int_fe['int_fe_2015'].drop('Afghanistan', axis = 1, inplace = True)
int_fe['int_fe_2016'].drop('Afghanistan', axis = 1, inplace = True)
data['integr_fe'] = int_fe

In [9]:
count_country = {}
for i in list(data.keys())[:7]:
    count_country[i] = len(list(data[i])) -1
    print('Кличество стран', i, ':', len(list(data[i])) -2)

Кличество стран 2013 : 149
Кличество стран 2014 : 228
Кличество стран 2015 : 228
Кличество стран 2016 : 228
Кличество стран 2017 : 127
Кличество стран 2018 : 126
Кличество стран 2019 : 128


In [10]:
max_country = {}
for key in list(data.keys())[:7]:
    if count_country[key] == max(count_country.values()):
        max_country[key] = max(count_country.values())
        break
print(max_country)

{'2014': 229}


In [11]:
err_country = []
region = pd.DataFrame({'country': list(data[list(max_country.keys())[0]])[3:], 'region': None})
for i in range(len(region.country)):
    try:
        region.region[i] = CountryInfo(region.country[i]).subregion()
    except:
        err_country.append(region.country[i])
region.drop_duplicates(inplace = True)

for i in err_country:
    print(i)
print('Количество нераспознанных стран:', len(err_country))

Andorra
Channel Islands
Curaçao
Vatican
Kosovo
Macao
Montenegro
Myanmar
Netherlands Antilles
Palestinian Territory, Occupied
Saint Martin (French part)
Sint Maarten (Dutch part)
Turks and Caicos Islands
Virgin Islands, British
Virgin Islands, U.S.
Количество нераспознанных стран: 15


In [12]:
#Заполняю в ручную
region.loc[region.country == err_country[0],['region']] = 'Western Europe' #Andorra
region.loc[region.country == err_country[1],['region']] = 'Western Europe' #Channel Islands
region.loc[region.country == err_country[2],['region']] = 'Caribbean' #Curaçao
region.loc[region.country == err_country[3],['region']] = 'Southern Europe' #Holy See (Vatican City State)
region.loc[region.country == err_country[4],['region']] = 'Eastern Europe' #Kosovo (used: Republic of Serbia)
region.loc[region.country == err_country[5],['region']] = 'Eastern Asia' #Macao (used: China)
region.loc[region.country == err_country[6],['region']] = 'Southern Europe' #Montenegro (used: Bosnia-Herzegovina)
region.loc[region.country == err_country[7],['region']] = 'Southern Asia' #Myanmar (used: Bangladesh)
region.loc[region.country == err_country[8],['region']] = 'South America' #Netherlands Antilles (used: Venezuela)
region.loc[region.country == err_country[9],['region']] = 'Western Asia' #Palestinian Territory, Occupied (used: Israel)
region.loc[region.country == err_country[10],['region']] = 'Caribbean' #Saint Martin (French part)
region.loc[region.country == err_country[11],['region']] = 'Caribbean' #Sint Maarten (Dutch part)
region.loc[region.country == err_country[12],['region']] = 'South America' #Turks and Caicos Islands (used: Cuba)
region.loc[region.country == err_country[13],['region']] = 'Caribbean' #Virgin Islands, British
region.loc[region.country == err_country[14],['region']] = 'Caribbean' #Virgin Islands, U.S.

data['region'] = region

In [13]:
gii = pd.DataFrame(data = None, columns = list(region['country']))

for file in file_list:
    gii_temp = pd.read_csv(os.path.join(data_dir, file))    
    gii_temp = gii_temp.loc[:1, 'Albania':'Zimbabwe']    
    gii_temp.drop([0], inplace = True)    
    gii_temp.rename(columns = rename_dict, inplace = True)    
    gii_temp = gii_temp.loc[:,~gii_temp.columns.duplicated()]    
    gii_temp.index.values[0] = re.sub('\D','',file)    
    gii = pd.concat([gii, gii_temp], sort=False)
    gii.fillna(gii.mean(), inplace = True)
    gii.fillna(0, inplace = True)

gii = gii.append(pd.Series(None, name='region'))
data['GII'] = gii

In [14]:
#код дополняет датасет region соотвествующим регионом страны
for i in list(data['region']['country']):
    data['GII'][i].region = list(data['region']['region'].loc[data['region']['country'] == i])[0]
data['GII']

D:\Program_Files\Anaconda\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
D:\Program_Files\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,"Virgin Islands, British","Virgin Islands, U.S.",Yemen,Zambia,Zimbabwe
2013,93,138,0,0,135,0,0,56,59,0,...,52,133,0,114,76,0,0,142,118,132
2014,94,133,0,0,135,0,0,70,65,0,...,72,128,0,122,71,0,0,141,121,130
2015,87,126,0,0,120,0,0,72,61,0,...,68,122,0,132,52,0,0,137,124,133
2016,92,113,0,0,130,0,0,81,60,0,...,62,127.667,0,120,59,0,0,128,125,131.667
2017,93,108,0,0,130,0,0,76,59,0,...,67,127.667,0,122,47,0,0,127,124,121
2018,83,110,0,0,130,0,0,80,68,0,...,62,127.667,0,122,45,0,0,126,120,113
2019,83,113,0,0,130,0,0,73,64,0,...,62,127.667,0,122,42,0,0,129,124,122
region,Southern Europe,Northern Africa,Polynesia,Western Europe,Middle Africa,Caribbean,Caribbean,South America,Western Asia,Caribbean,...,South America,Central Asia,Melanesia,South America,South-Eastern Asia,Caribbean,Caribbean,Western Asia,Eastern Africa,Eastern Africa


In [15]:
# Освобождаю память
del gii, gii_temp, region, max_country, count_country, indicators_temp, indicators_temp2, df_temp, err_country,  

In [16]:
#Данные Всемирного банка: worldbank.org
#'API_NY.GNP.MKTP.CD_DS2_en_excel_v2_1217953.xls' - csv не читается
data_dir = 'C:\\Users\\dserp\\ML\\Degree_ML\\GNI\\GNI_WB'
file_list = os.listdir(data_dir)

In [17]:
gni_wb = pd.read_excel(os.path.join(data_dir, file_list[0]))
gni_wb.head(4)  

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.250223e+09,2.302793e+09,2.418547e+09,2.537095e+09,2.662514e+09,2.794804e+09,2.830726e+09,2.885754e+09,NaN,NaN
1,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,5.488888e+08,5.600000e+08,5.577778e+08,7.666667e+08,8.155556e+08,1.026667e+09,...,1.588451e+10,1.778488e+10,2.003324e+10,2.063281e+10,2.048251e+10,2.008708e+10,1.954219e+10,2.044008e+10,1.967512e+10,1.940230e+10
2,Angola,AGO,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.571263e+10,1.020924e+11,1.176311e+11,1.268098e+11,1.368623e+11,1.102861e+11,9.585017e+10,1.146182e+11,9.352368e+10,8.719892e+10
3,Albania,ALB,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.180722e+10,1.289388e+10,1.222877e+10,1.299652e+10,1.335045e+10,1.152102e+10,1.205434e+10,1.305373e+10,1.512983e+10,1.512160e+10


In [18]:
#Привожу датаасет к виду gii
gni_wb = gni_wb.rename(index = gni_wb['Country Name'])

#удаляю столбцы 'Country Code', 'Indicator Name' и 'Indicator Code', а с 1960 по 2012
gni_wb.drop(gni_wb.columns[1:57], axis = 1, inplace = True) 
gni_wb.drop('Country Name', axis = 1, inplace = True)
gni_wb.dropna(axis = 0, how = 'all', inplace = True)
gni_wb = gni_wb.T

#Удаляю не нужные столюцы из qni_wb
gni_wb_drop = ['South Africa', 'East Asia & Pacific (IDA & IBRD countries)', 'Europe & Central Asia (IDA & IBRD countries)',
               'Latin America & the Caribbean (IDA & IBRD countries)', 'Middle East & North Africa (IDA & IBRD countries)',
               'South Asia (IDA & IBRD)','Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income', 'World', 'Middle East & North Africa',
               'Middle income','Middle East & North Africa (excluding high income)','OECD members','Other small states',
               'Pre-demographic dividend','West Bank and Gaza','Pacific island small states','Post-demographic dividend',
               'Sub-Saharan Africa (excluding high income)','Sub-Saharan Africa','Small states','Arab World',
               'Latin America & Caribbean (excluding high income)', 'Lao PDR','Least developed countries: UN classification',
               'Low income','Lower middle income','Low & middle income','Late-demographic dividend', 'East Asia & Pacific (excluding high income)',
               'Early-demographic dividend', 'East Asia & Pacific', 'Europe & Central Asia (excluding high income)', 'Europe & Central Asia', 'Egypt, Arab Rep.', 'Euro area', 'European Union',
               'Fragile and conflict affected situations', 'Micronesia, Fed. Sts.','Gambia, The','Hong Kong SAR, China','Heavily indebted poor countries (HIPC)','IBRD only',
               'IDA & IBRD total','IDA total','IDA blend','IDA only', 'High income','Afghanistan', 'Central Europe and the Baltics', 'Guinea',
               'Caribbean small states', 'Latin America & Caribbean','North America','South Asia','Eswatini', 'Timor-Leste']
gni_wb.drop(gni_wb_drop, axis = 1, inplace = True)

#переименовываю столбцы для большего соотвествия gii
gni_wb_rename = {"Cote d'Ivoire":'Ivory Coast', 'Bahamas, The' : 'Commonwealth of The Bahamas',
                 'Congo, Dem. Rep.':'Republic of the Congo','Congo, Rep.':'Republic of the Congo',
                 'Iran, Islamic Rep.':'Iran','Korea, Rep.':'Republic of Korea', 'Macao SAR, China':'Macao',
                 'Yemen, Rep.':'Yemen', 'United States':'United States of America', 
                 'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines', 'Brunei Darussalam':'Nation of Brunei',
                 'Cabo Verde' : 'Cape Verde', 'Curacao':'Curaçao','Guinea-Bissau':'Republic of Guinea-Bissau',
                 'Kyrgyz Republic':'Kyrgyzstan', 'St. Kitts and Nevis':'Saint Kitts and Nevis', 'St. Lucia':'Saint Lucia',
                 'North Macedonia':'Republic of Macedonia', 'Sao Tome and Principe' : "Democratic Republic of São Tomé and Príncipe",
                'Slovak Republic':'Slovakia', 'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines'}
gni_wb.rename(columns = gni_wb_rename, inplace = True)

#Заполняю пропуски средним значением по стране

nulls = gni_wb.isnull().sum()
for i in nulls[nulls > 0].keys():
    if i in nulls[nulls > 3]:
        gni_wb.drop(i, axis = 1, inplace = True)
    else:
        gni_wb[i] = gni_wb[i].fillna(gni_wb[i].median())
data['GNI'] = gni_wb

## Сохраяю датасет в pickle

In [19]:
import pickle
from datetime import datetime

In [20]:
#cохраняю датасет в файл
with open("C://Users//dserp//ML//Degree_ML//dataset_GW.pickle", 'wb') as f:
    pickle.dump(data, f)
print("Датасет сохранён:", datetime.fromtimestamp(os.path.getctime('C://Users//dserp//ML//Degree_ML//dataset_GW.pickle')).strftime('%Y-%m-%d %H:%M:%S'))
print('Размер датасета:', round(os.path.getsize('C://Users//dserp//ML//Degree_ML//dataset_GW.pickle')/1024, 2), 'Кб')
f.close()

Датасет сохранён: 2020-06-21 14:32:56
Размер датасета: 448.73 Кб


In [21]:
del data